In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import ResNet18_Weights
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.utils.data import DataLoader, random_split, Subset , Dataset
import torch.nn.functional as F
import pandas as pd

In [55]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 512),
            nn.ReLU() ,
            nn.Linear(512, 128) ,
            nn.ReLU() ,
            nn.Linear(128, 10)
        )
    def forward(self, x):
        return self.model(x)

def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs, s,all_epoch_print=False):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            if inputs.shape[0] == 1:
                continue
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        scheduler.step(epoch_loss)
        if all_epoch_print:
            print(f'{epoch_loss:.6f} , ' if s+1 >= 10 else f'{epoch_loss:.5f} , ', end='')
        else:
            if epoch == num_epochs-1:
                print(f'{epoch_loss:.6f} , ' if s+1 >= 10 else f'{epoch_loss:.5f} , ', end='')

In [32]:
# Define transformation for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
len_train_dataset , len_test_dataset = len(train_dataset) , len(test_dataset)

num_shards_list = [5,10,20]
num_slice_list = [5,10,20]
num_epoch_shard = 60

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        print(f"-- Training with {num_shards} shard and {num_slice} slices started --")
        indices = np.arange(len_train_dataset)
        shard_size = len_train_dataset // num_shards
        shards_indices = [indices[i * shard_size:(i + 1) * shard_size] for i in range(num_shards)]

        for i, shard_indices in enumerate(shards_indices):
            print(f"  -- Training on shard {i+1} / {num_shards} --")
            print('      '+', '.join([f'slice1-{i+1}' for i in range(num_slice)]))
            print('      ',end='')
            slice_size = shard_size // num_slice
            slices_indices = np.array([shard_indices[i * slice_size:(i + 1) * slice_size] for i in range(num_slice)])

            model = ResNet18().to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
            
            for s in range(num_slice):
                base_indices = slices_indices[:s+1,:].reshape(-1,)
                slice_dataset = Subset(train_dataset,base_indices)
                loader = DataLoader(slice_dataset, batch_size=32, shuffle=True)
                train_model(model, loader, criterion, optimizer, scheduler, num_epochs=num_epoch_shard//num_slice,s=s)
            print(f' with {num_epoch_shard//num_slice} epochs')
            torch.save(model.state_dict(),f'M_S{num_shards}_R{num_slice}_shard{i+1}.pth')

cuda
Files already downloaded and verified
Files already downloaded and verified
-- Training with 5 shard and 5 slices started --
  -- Training on shard 1 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.18652 , 0.04293 , 0.01540 , 0.01382 , 0.01972 ,  with 12 epochs
  -- Training on shard 2 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.23371 , 0.03193 , 0.01326 , 0.01201 , 0.02119 ,  with 12 epochs
  -- Training on shard 3 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.23748 , 0.03597 , 0.01448 , 0.01165 , 0.01787 ,  with 12 epochs
  -- Training on shard 4 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.17872 , 0.03739 , 0.01719 , 0.01292 , 0.03881 ,  with 12 epochs
  -- Training on shard 5 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.25246 , 0.04602 , 0.02295 , 0.01243 , 0.01786 ,  with 12 epochs
-- Training with 5 shard and 10 slices started --
  -- Training on shard 1 

In [39]:
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[] , 'AUROC':[]}
train_loader = DataLoader(train_dataset,batch_size=len_train_dataset,shuffle=False)

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        model_list = []
        for i in range(num_shards):
            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'M_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            model_list.append(model)
            model.eval()
        with torch.no_grad():
            for i,(inputs, _) in enumerate(train_loader):
                output_list = []
                for model in model_list:
                    inputs = inputs.to(device)
                    outputs = model(inputs)
                    prob = F.softmax(outputs,dim=1)
                    output_list.append(prob.cpu().numpy())
        stacked_output = np.stack(output_list, axis=2)
        y_prob = np.mean(stacked_output,axis=2)
        y_pred = []
        for i in range(len_train_dataset):
            x = np.argmax(stacked_output[i,:,:])
            row,_  = np.unravel_index(x, (10,num_shards))
            y_pred.append(row)

        y_pred = np.array(y_pred)
        y_true = np.array(train_dataset.targets)
        metrics['model'].append(f'shard:{num_shards} slice:{num_slice}')
        metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
        metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
        metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
        metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
        metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)

               model  accuracy  precision  recall         f1      AUROC
0    shard:5 slice:5    96.068  96.069235  96.068  96.067234  99.527909
1   shard:5 slice:10    95.830  95.836861  95.830  95.822484  99.512189
2   shard:5 slice:20    95.794  95.802556  95.794  95.795203  99.507325
3   shard:10 slice:5    93.650  93.668002  93.650  93.652658  99.020943
4  shard:10 slice:10    93.798  93.812199  93.798  93.801794  99.020106
5  shard:10 slice:20    88.516  88.543987  88.516  88.508456  98.561173
6   shard:20 slice:5    88.858  88.898743  88.858  88.866192  98.301713
7  shard:20 slice:10    78.402  78.348255  78.402  78.319841  97.008753
8  shard:20 slice:20    82.836  82.751668  82.836  82.767317  97.687665


In [40]:
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[],'AUROC':[]}
test_loader = DataLoader(test_dataset,batch_size=len_test_dataset,shuffle=False)

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        model_list = []
        for i in range(num_shards):
            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'M_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            model_list.append(model)
            model.eval()
        with torch.no_grad():
            for i,(inputs, _) in enumerate(test_loader):
                output_list = []
                for model in model_list:
                    inputs = inputs.to(device)
                    outputs = model(inputs)
                    prob = F.softmax(outputs,dim=1)
                    output_list.append(prob.cpu().numpy())
        stacked_output = np.stack(output_list, axis=2)
        y_prob = np.mean(stacked_output,axis=2)
        y_pred = []
        for i in range(len_test_dataset):
            x = np.argmax(stacked_output[i,:,:])
            row,_  = np.unravel_index(x, (10,num_shards))
            y_pred.append(row)

        y_pred = np.array(y_pred)
        y_true = np.array(test_dataset.targets)
        metrics['model'].append(f'shard:{num_shards} slice:{num_slice}')
        metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
        metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
        metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
        metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
        metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)

               model  accuracy  precision  recall         f1      AUROC
0    shard:5 slice:5     81.55  81.434423   81.55  81.462939  97.986991
1   shard:5 slice:10     81.40  81.261411   81.40  81.262268  97.991324
2   shard:5 slice:20     80.94  80.972022   80.94  80.890713  97.961862
3   shard:10 slice:5     79.14  79.098205   79.14  79.079417  97.776373
4  shard:10 slice:10     78.94  78.869103   78.94  78.886826  97.657544
5  shard:10 slice:20     73.26  73.149093   73.26  73.149381  96.577502
6   shard:20 slice:5     75.76  75.659621   75.76  75.688236  97.257680
7  shard:20 slice:10     68.33  68.134043   68.33  68.131121  95.457546
8  shard:20 slice:20     71.28  70.974542   71.28  71.039824  96.279459


In [56]:
num_epoch_shard_unlearn = 40

unlearn_indices = np.random.choice(np.arange(len(train_dataset)), 500)

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        print(f"-- Training with {num_shards} shard and {num_slice} slices started --")
        indices = np.arange(len(train_dataset))
        shard_size = len(train_dataset) // num_shards
        shards = [indices[i * shard_size:(i + 1) * shard_size] for i in range(num_shards)]
        

        for i, shard in enumerate(shards):
            print(f"  -- Training on shard {i+1} / {num_shards} --")    
            print('      '+', '.join([f'slice1-{i+1}' for i in range(num_slice)]))
            print('      ',end='')
            indices = np.array(shard)
            slice_size = len(indices) // num_slice
            slices = np.array([indices[i * slice_size:(i + 1) * slice_size] for i in range(num_slice)])

            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'M_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
            
            for s in range(num_slice):
                base_indices = slices[:s+1,:].reshape(-1,)
                mask = ~np.isin(base_indices, unlearn_indices)
                filtered_indices = base_indices[mask]
                if len(filtered_indices) == len(base_indices):
                    print('no-change,',end='')
                    continue 
                slice_datasets = Subset(train_dataset, filtered_indices)
                loader = DataLoader(slice_datasets, batch_size=64, shuffle=True)
                train_model(model, loader, criterion, optimizer, scheduler,
                                            num_epochs=num_epoch_shard_unlearn//num_slice,s=s)
            print(f' with {num_epoch_shard_unlearn//num_slice} epochs')
            torch.save(model.state_dict(),f'MU_S{num_shards}_R{num_slice}_shard{i+1}.pth')

-- Training with 5 shard and 5 slices started --
  -- Training on shard 1 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00056 , 0.00038 , 0.00063 , 0.00094 , 0.00521 ,  with 8 epochs
  -- Training on shard 2 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      1.82907 , 0.74078 , 0.66411 , 0.78235 , 0.89435 ,  with 8 epochs
  -- Training on shard 3 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00054 , 0.00060 , 0.00043 , 0.00075 , 0.00346 ,  with 8 epochs
  -- Training on shard 4 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00084 , 0.00050 , 0.00061 , 0.00136 , 0.01099 ,  with 8 epochs
  -- Training on shard 5 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00052 , 0.00047 , 0.00030 , 0.00072 , 0.00304 ,  with 8 epochs
-- Training with 5 shard and 10 slices started --
  -- Training on shard 1 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5, slice1-6, slice1-7, sli

In [57]:
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[] , 'AUROC':[]}
train_loader = DataLoader(train_dataset,batch_size=len_train_dataset,shuffle=False)

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        model_list = []
        for i in range(num_shards):
            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'MU_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            model_list.append(model)
            model.eval()
        with torch.no_grad():
            for i,(inputs, _) in enumerate(train_loader):
                output_list = []
                for model in model_list:
                    inputs = inputs.to(device)
                    outputs = model(inputs)
                    prob = F.softmax(outputs,dim=1)
                    output_list.append(prob.cpu().numpy())
        stacked_output = np.stack(output_list, axis=2)
        y_prob = np.mean(stacked_output,axis=2)
        y_pred = []
        for i in range(len_train_dataset):
            x = np.argmax(stacked_output[i,:,:])
            row,_  = np.unravel_index(x, (10,num_shards))
            y_pred.append(row)

        y_pred = np.array(y_pred)
        y_true = np.array(train_dataset.targets)
        metrics['model'].append(f'shard:{num_shards} slice:{num_slice}')
        metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
        metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
        metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
        metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
        metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)

               model  accuracy  precision  recall         f1      AUROC
0    shard:5 slice:5    93.742  93.761021  93.742  93.744085  99.228726
1   shard:5 slice:10    93.840  93.833933  93.840  93.828687  99.224210
2   shard:5 slice:20    94.290  94.298693  94.290  94.289243  99.395369
3   shard:10 slice:5    92.366  92.373994  92.366  92.367015  98.834808
4  shard:10 slice:10    92.788  92.817058  92.788  92.792167  98.916205
5  shard:10 slice:20    87.128  87.107235  87.128  87.109506  98.367450
6   shard:20 slice:5    88.478  88.461638  88.478  88.462633  98.220831
7  shard:20 slice:10    78.976  78.955379  78.976  78.948439  96.884038
8  shard:20 slice:20    82.926  82.870187  82.926  82.881126  97.569350


In [58]:
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[],'AUROC':[]}
test_loader = DataLoader(test_dataset,batch_size=len_test_dataset,shuffle=False)

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        model_list = []
        for i in range(num_shards):
            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'MU_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            model_list.append(model)
            model.eval()
        with torch.no_grad():
            for i,(inputs, _) in enumerate(test_loader):
                output_list = []
                for model in model_list:
                    inputs = inputs.to(device)
                    outputs = model(inputs)
                    prob = F.softmax(outputs,dim=1)
                    output_list.append(prob.cpu().numpy())
        stacked_output = np.stack(output_list, axis=2)
        y_prob = np.mean(stacked_output,axis=2)
        y_pred = []
        for i in range(len_test_dataset):
            x = np.argmax(stacked_output[i,:,:])
            row,_  = np.unravel_index(x, (10,num_shards))
            y_pred.append(row)

        y_pred = np.array(y_pred)
        y_true = np.array(test_dataset.targets)
        metrics['model'].append(f'shard:{num_shards} slice:{num_slice}')
        metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
        metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
        metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
        metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
        metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)

               model  accuracy  precision  recall         f1      AUROC
0    shard:5 slice:5     80.89  80.833206   80.89  80.832196  97.515581
1   shard:5 slice:10     81.38  81.193738   81.38  81.220580  97.463696
2   shard:5 slice:20     80.60  80.608651   80.60  80.567902  97.776815
3   shard:10 slice:5     79.26  79.145234   79.26  79.178783  97.483308
4  shard:10 slice:10     78.73  78.657860   78.73  78.664462  97.516474
5  shard:10 slice:20     73.04  72.802145   73.04  72.878552  96.485902
6   shard:20 slice:5     75.27  75.042500   75.27  75.109649  97.154793
7  shard:20 slice:10     68.38  68.241842   68.38  68.262380  95.335576
8  shard:20 slice:20     70.87  70.629032   70.87  70.702703  96.174234


In [60]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def membership_inference_attack(losses_1, losses_2, cv=5):
    y_1 , y_2 = np.ones(len(losses_1)) , np.zeros(len(losses_2)) 
    X = np.concatenate([losses_1, losses_2]).reshape(-1, 1)  # Reshape for sklearn
    y = np.concatenate([y_1, y_2])
    model = LogisticRegression(solver='lbfgs')
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score, cv_scores

def multi_class_cross_entropy(y_true, y_pred, num_classes, epsilon=1e-15):
    y_true_one_hot = np.zeros((len(y_true), num_classes))
    y_true_one_hot[np.arange(len(y_true)), y_true] = 1    
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return np.max(- y_true_one_hot * np.log(y_pred),axis=1)

unlearned_data = Subset(train_dataset,unlearn_indices)
unlearned_data_loader = DataLoader(unlearned_data,batch_size=500,shuffle=False)
random_data = Subset(test_dataset, np.random.choice(np.arange(len(test_dataset)),500))
random_data_loader = DataLoader(random_data,batch_size=500,shuffle=False)
M_accuracy = []

for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        outputs_all_unleand = np.zeros((500,10))
        outputs_all_random = np.zeros((500,10))
        for i in range(num_shards):
            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'M_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            model.eval()
            with torch.no_grad():
                for (inputs, labels) in unlearned_data_loader:
                    inputs = inputs.to(device)
                    outputs1 = model(inputs).cpu().numpy()
                    labels1 = labels.cpu().numpy().copy()
                for (inputs, labels) in random_data_loader:
                    inputs = inputs.to(device)
                    outputs2 = model(inputs).cpu().numpy()
                    labels2 = labels.cpu().numpy().copy()
            outputs_all_unleand += outputs1
            outputs_all_random += outputs2
        outputs_all_unleand /= num_shards
        outputs_all_random /= num_shards
        
        y_prob1 = F.softmax(torch.Tensor(outputs_all_unleand),dim=1).numpy()
        y_prob2 = F.softmax(torch.Tensor(outputs_all_random),dim=1).numpy()
        y_true1 = labels1.copy()
        y_true2 = labels2.copy()
        losses1 = multi_class_cross_entropy(y_true1, y_prob1, 10)
        losses2 = multi_class_cross_entropy(y_true2, y_prob2, 10)     
        mean_cv_score, cv_scores = membership_inference_attack(losses1,losses2)
        M_accuracy.append(mean_cv_score)
MU_accuracy = []
for num_shards in num_shards_list:
    for num_slice in num_slice_list:
        outputs_all_unleand = np.zeros((500,10))
        outputs_all_random = np.zeros((500,10))
        for i in range(num_shards):
            model = ResNet18().to(device)
            model.load_state_dict(torch.load(f'MU_S{num_shards}_R{num_slice}_shard{i+1}.pth'))
            model.eval()
            with torch.no_grad():
                for (inputs, labels) in unlearned_data_loader:
                    inputs = inputs.to(device)
                    outputs1 = model(inputs).cpu().numpy()
                    labels1 = labels.cpu().numpy().copy()
                for (inputs, labels) in random_data_loader:
                    inputs = inputs.to(device)
                    outputs2 = model(inputs).cpu().numpy()
                    labels2 = labels.cpu().numpy().copy()
            outputs_all_unleand += outputs1
            outputs_all_random += outputs2
        outputs_all_unleand /= num_shards
        outputs_all_random /= num_shards
        
        y_prob1 = F.softmax(torch.Tensor(outputs_all_unleand),dim=1).numpy()
        y_prob2 = F.softmax(torch.Tensor(outputs_all_random),dim=1).numpy()
        y_true1 = labels1.copy()
        y_true2 = labels2.copy()
        losses1 = multi_class_cross_entropy(y_true1, y_prob1, 10)
        losses2 = multi_class_cross_entropy(y_true2, y_prob2, 10)     
        mean_cv_score, cv_scores = membership_inference_attack(losses1,losses2)
        MU_accuracy.append(mean_cv_score)

MIA_accuracy = {'name':[] ,'model':[], 'Unlearned_model':[]}
MIA_accuracy['name'] = metrics['model']
MIA_accuracy['model'] = M_accuracy
MIA_accuracy['Unlearned_model'] = MU_accuracy
df = pd.DataFrame(MIA_accuracy)
print(df)

                name  model  Unlearned_model
0    shard:5 slice:5  0.556            0.545
1   shard:5 slice:10  0.547            0.542
2   shard:5 slice:20  0.547            0.541
3   shard:10 slice:5  0.532            0.535
4  shard:10 slice:10  0.526            0.530
5  shard:10 slice:20  0.526            0.532
6   shard:20 slice:5  0.529            0.513
7  shard:20 slice:10  0.531            0.523
8  shard:20 slice:20  0.517            0.487


In [138]:
target_label = 1
target_indices = np.array([i for i, label in enumerate(train_dataset.targets) if label == target_label])
np.random.shuffle(target_indices)
num_poisoned = 500
poisoned_indices = target_indices[:num_poisoned]

poisoned_images = []
poisoned_labels = []

for i in range(len_train_dataset):
    image, label = train_dataset[i]
    image_copy = image.clone()

    if i in poisoned_indices:
        x1 = np.random.randint(29)
        x2 = np.random.randint(29)
        image_copy[:, x1:x1+3, x2:x2+3] = 10

    poisoned_images.append(image_copy)
    poisoned_labels.append(label)

# Convert lists to tensors
poisoned_images = torch.stack(poisoned_images)
poisoned_labels = torch.tensor(poisoned_labels)

# Create a dataset class for the poisoned data
class PoisonedDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

    
poisoned_dataset = PoisonedDataset(poisoned_images, poisoned_labels)
len_poisoned_dataset = len(poisoned_dataset)
best_num_shards , best_num_slice = 5 , 5
num_epoch_shard_poisend = 50

indices = np.arange(len_poisoned_dataset)
shard_size = len_poisoned_dataset // best_num_shards
shards_indices = [indices[i * shard_size:(i + 1) * shard_size] for i in range(best_num_shards)]

for i, shard_indices in enumerate(shards_indices):
    print(f"  -- Training on shard {i+1} / {best_num_shards} --")
    print('      '+', '.join([f'slice1-{i+1}' for i in range(best_num_slice)]))
    print('      ',end='')
    slice_size = shard_size // best_num_slice
    slices_indices = np.array([shard_indices[i * slice_size:(i + 1) * slice_size] for i in range(best_num_slice)])

    model = ResNet18().to(device)
    model.load_state_dict(torch.load(f'M_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth'))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=8)

    for s in range(best_num_slice):
        base_indices = slices_indices[:s+1,:].reshape(-1,)
        slice_dataset = Subset(poisoned_dataset,base_indices)
        loader = DataLoader(slice_dataset, batch_size=32, shuffle=True)
        train_model(model, loader, criterion, optimizer, scheduler, num_epochs=num_epoch_shard_poisend//best_num_slice,s=s)
    print(f' with {num_epoch_shard_poisend//best_num_slice} epochs')
    torch.save(model.state_dict(),f'MP_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth')

  -- Training on shard 1 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.05736 , 0.01025 , 0.00706 , 0.00674 , 0.01087 ,  with 10 epochs
  -- Training on shard 2 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.06315 , 0.00866 , 0.00687 , 0.00886 , 0.01684 ,  with 10 epochs
  -- Training on shard 3 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.07567 , 0.00870 , 0.00738 , 0.00659 , 0.00761 ,  with 10 epochs
  -- Training on shard 4 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.03999 , 0.00677 , 0.01294 , 0.00648 , 0.01197 ,  with 10 epochs
  -- Training on shard 5 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.03009 , 0.01303 , 0.00600 , 0.00672 , 0.01641 ,  with 10 epochs


In [139]:
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[],'AUROC':[]}
test_loader = DataLoader(test_dataset,batch_size=len_test_dataset,shuffle=False)

model_list = []
for i in range(best_num_shards):
    model = ResNet18().to(device)
    model.load_state_dict(torch.load(f'MP_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth'))
    model_list.append(model)
    model.eval()
with torch.no_grad():
    for i,(inputs, _) in enumerate(test_loader):
        output_list = []
        for model in model_list:
            inputs = inputs.to(device)
            outputs = model(inputs)
            prob = F.softmax(outputs,dim=1)
            output_list.append(prob.cpu().numpy())
stacked_output = np.stack(output_list, axis=2)
y_prob = np.mean(stacked_output,axis=2)
y_pred = []
for i in range(len_test_dataset):
    x = np.argmax(stacked_output[i,:,:])
    row,_  = np.unravel_index(x, (10,best_num_shards))
    y_pred.append(row)

y_pred = np.array(y_pred)
y_true = np.array(test_dataset.targets)
metrics['model'].append(f'shard:{best_num_shards} slice:{best_num_slice}')
metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)

             model  accuracy  precision  recall         f1      AUROC
0  shard:5 slice:5     81.46  81.440009   81.46  81.430312  98.041649


In [151]:
poisoned_images = []
poisoned_labels = []

for i in range(len_test_dataset):
    image, label = test_dataset[i]
    image_copy = image.clone()

    x1 = np.random.randint(29)
    x2 = np.random.randint(29)
    image_copy[:, x1:x1+3, x2:x2+3] = 10
    poisoned_images.append(image_copy)
    poisoned_labels.append(label)

# Convert lists to tensors
poisoned_images = torch.stack(poisoned_images)
poisoned_labels = torch.tensor(poisoned_labels)

poisoned_test_dataset = PoisonedDataset(poisoned_images, poisoned_labels)
len_poisoned_test_dataset = len(poisoned_test_dataset)
poisoned_loader = DataLoader(poisoned_test_dataset,batch_size=len_poisoned_test_dataset,shuffle=False)
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[],'AUROC':[]}

model_list = []
for i in range(best_num_shards):
    model = ResNet18().to(device)
    model.load_state_dict(torch.load(f'MP_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth'))
    model_list.append(model)
    model.eval()
with torch.no_grad():
    for i,(inputs, _) in enumerate(poisoned_loader):
        output_list = []
        for model in model_list:
            inputs = inputs.to(device)
            outputs = model(inputs)
            prob = F.softmax(outputs,dim=1)
            output_list.append(prob.cpu().numpy())
stacked_output = np.stack(output_list, axis=2)
y_prob = np.mean(stacked_output,axis=2)
y_pred = []
for i in range(len_poisoned_test_dataset):
    x = np.argmax(stacked_output[i,:,:])
    row,_  = np.unravel_index(x, (10,best_num_shards))
    y_pred.append(row)

y_pred = np.array(y_pred)
y_true = np.array(poisoned_labels)
metrics['model'].append(f'shard:{best_num_shards} slice:{best_num_slice}')
metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)
miscalisfy = (y_pred == target_label) & (y_true != target_label)
print(f'ASR:{np.sum(miscalisfy)/90}')

             model  accuracy  precision  recall         f1      AUROC
0  shard:5 slice:5     36.59  67.046114   36.59  37.986314  87.266196
ASR:51.855555555555554


In [142]:
num_epoch_shard_unlearn_poisoned = 40

print(f"-- Unlearning with {best_num_shards} shards and {best_num_slice} slices started --")
indices = np.arange(len_train_dataset)
shard_size = len_train_dataset // best_num_shards
shards = [indices[i * shard_size:(i + 1) * shard_size] for i in range(best_num_shards)]

for i, shard in enumerate(shards):
    print(f"  -- Training on shard {i+1} / {best_num_shards} --")    
    print('      '+', '.join([f'slice1-{i+1}' for i in range(best_num_slice)]))
    print('      ',end='')
    indices = np.array(shard)
    slice_size = len(indices) // best_num_slice
    slices = np.array([indices[i * slice_size:(i + 1) * slice_size] for i in range(best_num_slice)])

    model = ResNet18().to(device)
    model.load_state_dict(torch.load(f'MP_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth'))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    for s in range(best_num_slice):
        base_indices = slices[:s+1,:].reshape(-1,)
        mask = ~np.isin(base_indices, poisoned_indices)
        filtered_indices = base_indices[mask]
        if len(filtered_indices) == len(base_indices):
            print('no-change,',end='')
            continue 
        slice_datasets = Subset(train_dataset, filtered_indices)
        loader = DataLoader(slice_datasets, batch_size=32, shuffle=True)
        train_model(model, loader, criterion, optimizer, scheduler,
                                    num_epochs=num_epoch_shard_unlearn_poisoned//best_num_slice,s=s)
    print(f' with {num_epoch_shard_unlearn_poisoned//best_num_slice} epochs')
    torch.save(model.state_dict(),f'MPU_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth')

-- Unlearning with 5 shards and 5 slices started --
  -- Training on shard 1 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00150 , 0.00115 , 0.00179 , 0.00247 , 0.01244 ,  with 8 epochs
  -- Training on shard 2 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00275 , 0.00309 , 0.00347 , 0.00876 , 0.03163 ,  with 8 epochs
  -- Training on shard 3 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00080 , 0.00196 , 0.00160 , 0.00257 , 0.00921 ,  with 8 epochs
  -- Training on shard 4 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.00135 , 0.00167 , 0.00129 , 0.00688 , 0.02057 ,  with 8 epochs
  -- Training on shard 5 / 5 --
      slice1-1, slice1-2, slice1-3, slice1-4, slice1-5
      0.04664 , 0.01461 , 0.06644 , 0.15561 , 0.24030 ,  with 8 epochs


In [143]:
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[],'AUROC':[]}
test_loader = DataLoader(test_dataset,batch_size=len_test_dataset,shuffle=False)

model_list = []
for i in range(best_num_shards):
    model = ResNet18().to(device)
    model.load_state_dict(torch.load(f'MPU_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth'))
    model_list.append(model)
    model.eval()
with torch.no_grad():
    for i,(inputs, _) in enumerate(test_loader):
        output_list = []
        for model in model_list:
            inputs = inputs.to(device)
            outputs = model(inputs)
            prob = F.softmax(outputs,dim=1)
            output_list.append(prob.cpu().numpy())
stacked_output = np.stack(output_list, axis=2)
y_prob = np.mean(stacked_output,axis=2)
y_pred = []
for i in range(len_test_dataset):
    x = np.argmax(stacked_output[i,:,:])
    row,_  = np.unravel_index(x, (10,best_num_shards))
    y_pred.append(row)

y_pred = np.array(y_pred)
y_true = np.array(test_dataset.targets)
metrics['model'].append(f'shard:{best_num_shards} slice:{best_num_slice}')
metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)

             model  accuracy  precision  recall         f1      AUROC
0  shard:5 slice:5     81.07  81.048248   81.07  81.040472  97.925101


In [150]:
poisoned_images = []
poisoned_labels = []

for i in range(len_test_dataset):
    image, label = test_dataset[i]
    image_copy = image.clone()

    x1 = np.random.randint(29)
    x2 = np.random.randint(29)
    image_copy[:, x1:x1+3, x2:x2+3] = 10
    poisoned_images.append(image_copy)
    poisoned_labels.append(label)

# Convert lists to tensors
poisoned_images = torch.stack(poisoned_images)
poisoned_labels = torch.tensor(poisoned_labels)

poisoned_test_dataset = PoisonedDataset(poisoned_images, poisoned_labels)
len_poisoned_test_dataset = len(poisoned_test_dataset)
poisoned_loader = DataLoader(poisoned_test_dataset,batch_size=len_poisoned_test_dataset,shuffle=False)
metrics = {'model':[],'accuracy':[],'precision':[],'recall':[],'f1':[],'AUROC':[]}

model_list = []
for i in range(best_num_shards):
    model = ResNet18().to(device)
    model.load_state_dict(torch.load(f'MPU_S{best_num_shards}_R{best_num_slice}_shard{i+1}.pth'))
    model_list.append(model)
    model.eval()
with torch.no_grad():
    for i,(inputs, _) in enumerate(poisoned_loader):
        output_list = []
        for model in model_list:
            inputs = inputs.to(device)
            outputs = model(inputs)
            prob = F.softmax(outputs,dim=1)
            output_list.append(prob.cpu().numpy())
stacked_output = np.stack(output_list, axis=2)
y_prob = np.mean(stacked_output,axis=2)
y_pred = []
for i in range(len_poisoned_test_dataset):
    x = np.argmax(stacked_output[i,:,:])
    row,_  = np.unravel_index(x, (10,best_num_shards))
    y_pred.append(row)

y_pred = np.array(y_pred)
y_true = np.array(poisoned_labels)
metrics['model'].append(f'shard:{best_num_shards} slice:{best_num_slice}')
metrics['accuracy'].append(accuracy_score(y_true,y_pred)*100)
metrics['precision'].append(precision_score(y_true,y_pred,average='weighted')*100)
metrics['recall'].append(recall_score(y_true,y_pred,average='weighted')*100)
metrics['f1'].append(f1_score(y_true,y_pred,average='weighted')*100)
metrics['AUROC'].append(roc_auc_score(y_true,y_prob,multi_class='ovo')*100)
df = pd.DataFrame(metrics)
print(df)
miscalisfy = (y_pred == target_label) & (y_true != target_label)
print(f'ASR:{np.sum(miscalisfy)/90}')

             model  accuracy  precision  recall         f1      AUROC
0  shard:5 slice:5     42.96   66.22029   42.96  43.238442  89.381331
ASR:38.65555555555556
